This notebook explores the use of the permutation test to assess the significance of coefficents learned in logistic regression (testing against the null that each $\beta$ = 0).

In [ ]:
import sys
from sklearn import preprocessing
from sklearn import linear_model
from random import choices
from sklearn.feature_extraction.text import CountVectorizer
from random import shuffle
import numpy as np
import copy

In [ ]:
def read_data(filename):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            label=cols[0]
            text=cols[1]
            # assumes text is already tokenized
            X.append(text)
            Y.append(label)
    return X, Y

In [ ]:
# Change this to the directory with your data (from the CheckData_TODO.ipynb exercise).  
# The directory should contain train.tsv, dev.tsv and test.tsv
directory="../data/text_classification_sample_data"

In [ ]:
trainX, trainY=read_data("%s/train.tsv" % directory)
devX, devY=read_data("%s/dev.tsv" % directory)

In [ ]:
def featurize(trainX, devX):
    vectorizer = CountVectorizer(max_features=10000, analyzer=str.split, lowercase=False, strip_accents=None, binary=True)

    X_train = vectorizer.fit_transform(trainX)
    X_dev = vectorizer.transform(devX)

    return X_train, X_dev, vectorizer

In [ ]:
def train(X_train, trainY, le):
    Y_train=le.transform(trainY)
    logreg = linear_model.LogisticRegression(C=100, solver='lbfgs', penalty='l2', max_iter=10000)
    logreg.fit(X_train, Y_train)
    return logreg
    return logreg.coef_[0]

In [ ]:
def test(logreg, devX_feat, devY, le):
    Y_dev=le.transform(devY)
    print("Accuracy: %.3f" % logreg.score(devX_feat, Y_dev))

In [ ]:
def analyze_weights(coefs, label_encoder, vocab, p_values):
    reverse_vocab = {v: k for k, v in vocab.items()}

    sort_index = np.argsort(coefs)

    print(label_encoder.inverse_transform([0])[0])
    for k in sort_index[:25]:
        print ("%.5f\t%s\t%.4f" % (coefs[k], reverse_vocab[k], p_values[k] ))

    print(label_encoder.inverse_transform([1])[0])

    for k in reversed(sort_index[-25:]):
        print ("%.5f\t%s\t%.4f" % (coefs[k], reverse_vocab[k], p_values[k] ))

In [ ]:
X_train, X_dev, vectorizer=featurize(trainX, devX)
le = preprocessing.LabelEncoder()
le.fit(trainY)

logreg=train(X_train, trainY, le)
test(logreg, X_dev, devY, le)

true_coefficients=logreg.coef_[0]

# We'll set P=100 here to finish running in class, but set higher (e.g., 10000) for real applications
P=100

p_values=np.zeros(len(true_coefficients))
permutedY=copy.deepcopy(trainY)

for i in range(P):
    if i % 10 == 0:
        print(i)
    
    # permute the values of Y so that they're now attached to random data points in X
    shuffle(permutedY)
    
    # train logistic regression on that permuted dataset
    permuted_logreg=train(X_train, permutedY, le)
    coefficients=permuted_logreg.coef_[0]
    
    # test how often the coefficients learned from the permuted data are as extreme as
    # the coefficients from the true data
    for idx, coef in enumerate(coefficients):
        if abs(true_coefficients[idx]) < abs(coef):
            p_values[idx]+=1./P

In [ ]:
inverse_vocab = {v: k for k, v in vectorizer.vocabulary_.items()}            
out=open("weights.txt", "w")
for idx, coef in enumerate(true_coefficients):
    out.write("%.3f\t%s\t%.5f\n" % (coef, inverse_vocab[idx], p_values[idx]))
out.close()

In [ ]:
analyze_weights(true_coefficients, le, vectorizer.vocabulary_, p_values)